<h2>Save and load model</h2>
<p>The model can be saved during and after training. That is a model can resume where it left off and avoid long training times. Saving also means we can share the model to others and they can recreate our work with different data. When publishing research models and techniques, most machine learning practitioners share code to create the model and the trained weights or parameters of the model. Sharing of this model data help others to understand the working of model and try themselves with new data.</p>  

<p>There are several ways to save the TensorFlow models depending the functional API we're using. We use tf.keras, high level API to build and train the models in TensorFlow. <p>

In [1]:
#Importing required libraries
import os

import tensorflow as tf
from tensorflow import keras


<h4>The Example Dataset</h4>
<p>To show how to save and load model we use MNIST dataset. We are only taking only 1000 training ang test samples.</p>

In [2]:
# Loading the dataset using keras
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

#Taking only 1000 samples for training and testing
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

# Processing the images in the dataset
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


<h4>Define a model</h4>

In [3]:
# Defining a function for creating a simple sequential model and compiling the model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

#Compiling the model with adam optimizer
  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


<h3>Save checkpoints during training</h3>
<p> We can pick up the training where the model left-off and trained model. The callback <b>tf.keras.callbacks.ModelCheckpoint </b> allows us to continually save the model in both during and at the end of training.</p>
<h4>Checkpoint callback usage</h4>
<p>Create a tf.keras.callbacks.ModelCheckpoint callback that saves weight only during training.</p> 

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
32/32 [==============================] - 1s 24ms/step - loss: 1.7267 - sparse_categorical_accuracy: 0.4505 - val_loss: 0.7423 - val_sparse_categorical_accuracy: 0.7670

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 9ms/step - loss: 0.4939 - sparse_categorical_accuracy: 0.8632 - val_loss: 0.5243 - val_sparse_categorical_accuracy: 0.8360

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 10ms/step - loss: 0.2813 - sparse_categorical_accuracy: 0.9372 - val_loss: 0.4831 - val_sparse_categorical_accuracy: 0.8510

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 9ms/step - loss: 0.2454 - sparse_categorical_accuracy: 0.9344 - val_loss: 0.4497 - val_sparse_categorical_accuracy: 0.8560

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 8ms/step - loss: 0.1485 - sparse_ca

<p>This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch.</p>

In [5]:
# list the directories in checkpoint_dir
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'cp.ckpt.data-00000-of-00001', 'checkpoint']

<p>If two models share the same architecture we can share weights between them. So, when restoring a model from weights-only, create a model with same architecture as the original model and then set its weights. We will build the model, untrained and evaluate it on test set. An untrained model will perform at chance levels(approximate 10% accuracy).</p>

In [6]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.4393 - sparse_categorical_accuracy: 0.0520
Untrained model, accuracy:  5.20%


<p>Load the weights from the checkpoints and re-evaluate the model</p>

In [7]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4278 - sparse_categorical_accuracy: 0.8610
Restored model, accuracy: 86.10%


<h4>Checkpoint callback options</h4>
<p>The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency. We train a new model and save uniquely named checkpoints once every 10 epochs</p>

In [8]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=10*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


<p>Look at the checkpoints and choose the latest one</p>

In [9]:
# Display the checkpoint list
os.listdir(checkpoint_dir)

['cp-0030.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index']

In [10]:
# Print the latest checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

<p>To test, reset the model and laod the latest checkpoint</p>

In [11]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4826 - sparse_categorical_accuracy: 0.8790
Restored model, accuracy: 87.90%


<h3>What are these files?</h3>
<p>The above experimented code store the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. These checkpoint contains:</p>
<ul><li>One or more shards that contain your model's weights</li>
<li>An index file that indicates which weights are stored in which shard</li></ul>
<p>If we are training the model on single machine, we'll have one shard with the suffix <b>.data-00000-of-00001</p>


<h3>Manually save weights</h3>
<p><b>Model.save_weights</b> method will save the weights manually. By default, tf.keras and save_weights in particular uses the TensorFlow checkpoint format with a .ckpt extension. Weights can also saved in HDF5 with .h5 extension.</p> 

In [12]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4826 - sparse_categorical_accuracy: 0.8790
Restored model, accuracy: 87.90%


<h3>Save the entire model</h3>
<p>The method called <b>model.save</b> will save the model's architecture, weights and training configuration in a single file or folder. This will allow us to export a model so it can be used without accesss to the original python code. Since the optimizer-state is recovered, we can resume training from exactly where we left. An entire model can be saved in two different formats. The TensorFlow <b>savedModel</b> format is the default file format in TF2.x. Another way is saving in HDF5 format.<br>
Saving a fully functional model is very useful, we can load them in <b>TensorFlow.js(Saved Model, HDF5)</b>. and then train and run them in web browsers or convert them to run on mobile devices using <b>TensorFlow Lite(Saved Model, HDF5)</b>. Custom objects (example, subclassed models or layers) require special attention when saving and loading.</p> 

<h4>SavedModel format<//h4>
<p>The saved model format is another way to serialize models. Models saved in this format can be restored using <b>tf.keras.models.load_model</b> and they are compatible with TensorFlow serving. The below section illustrates the steps to save and restore the model.</p>

In [13]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.6002 - sparse_categorical_accuracy: 0.5273
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4240 - sparse_categorical_accuracy: 0.8868
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2716 - sparse_categorical_accuracy: 0.9284
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1899 - sparse_categorical_accuracy: 0.9606
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1443 - sparse_categorical_accuracy: 0.9770
INFO:tensorflow:Assets written to: saved_model/my_model/assets


<p> The SavedModel format is a directory consisting a protobuf binary and a TensorFlow checkpoint. We reload a keras model from the saved model in the below section.</p> 

In [14]:
# Loading the saved model to the new model
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


<p>The restored model is compiled with same arguements as the original model. We try to run, evaluate and predict the loaded model.</p>

In [15]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4178 - sparse_categorical_accuracy: 0.8680
Restored model, accuracy: 86.80%
(1000, 10)


<h4>HDF5 format</h4>
<p>Keras provide a basic save format using the HDF5 standard.</h4>

In [16]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.5763 - sparse_categorical_accuracy: 0.4933
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4323 - sparse_categorical_accuracy: 0.8810
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.3470 - sparse_categorical_accuracy: 0.9062
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2225 - sparse_categorical_accuracy: 0.9411
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1588 - sparse_categorical_accuracy: 0.9645


<p>Recreating the model using .h5 extension file</p>

In [17]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Check the accuracy of the model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4409 - sparse_categorical_accuracy: 0.8580
Restored model, accuracy: 85.80%


<p>Keras saves models by inspecting their architectures. This techniques saves the following items:</p>
<ul><li>The weight values</li>
<li>The model's architecture</li>
<li>The model's trainng configuration (what we pass to the .compile() method)</li>
<li>The optimizer and its state</li></ul>
<p> Keras is not able to save the <b>v1.x</b> optimizers (these are from tf.compat.v1.train). They aren't compatible with checkpoints. For v1.x optimizers, we need to re-compile the model after loading-losing the state of the optimizer.</p>


<h4>Saving custom objects</h4>
<p> The main difference between HDF5 and SavedModels is that HDF5 uses object configs to save the model architecture, while SavedModel save the execution graphs. Thus SavedModel are able to save custom objects like subclassed models and custom layers without requiring the original code. To save custom objects to HDF5, we must do the following steps:</p>
<ul><li>Define a <b>get_config</b> method in our object, and optionally a <b>from_config</b> classmethod.
<ul><li>get_config(self) returns a JSON-serializable dictionary of parameters needed to recreate the project.</li>
<li>from_config(cls, config) uses the returned config from get_config to create a new object. By default, this function will use the config as ininitialization kwargs(return cls(**config)).</li></ul></li>
<li>Pass the object to the custom_objects argument when loading the model. The argument must be a dictionary mapping the string class name to the python class.Example, tf.keras.models.load_model(path, custom_objects = { 'CustomLayer':CustomLayer})</li></ul>